# 転移学習用のデータ保存

In [ ]:
import io
import os
import pickle
import time

import kachaka_api
import numpy as np
from IPython.display import Image, display
from PIL import Image as PILImage

In [ ]:
client = kachaka_api.KachakaApiClient()

os.makedirs("data", exist_ok=True)
data_num = 0

撮影と保存 (繰り返し実行して下さい)

In [ ]:
now = time.time()

while True:
    image = client.get_front_camera_ros_compressed_image()
    features_header, features = client.get_object_detection_features()
    if now < image.header.stamp_nsec / 10e8 and now < features_header.stamp_nsec / 10e8:
        break

inputs = {
    feature.name.replace("_out", ""): np.array(feature.data, dtype=np.float32).reshape(
        feature.shape
    )
    for feature in features
}

with open(f"data/{data_num}.pkl", "wb") as f:
    pickle.dump(inputs, f)

pil_image = PILImage.open(io.BytesIO(image.data))
pil_image.save(f"data/{data_num}.png")

Image(data=image.data, format="jpeg")
display(Image(data=image.data, format="jpeg"))

data_num += 1

データ保存後、転移学習を行う方法については以下READMEをご覧下さい https://github.com/pf-robotics/kachaka-transfer-learning